# Assignment 2: The Winter is here
##### This works best with epic battle music. No spoilers present.
<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Tywin Lannister was right when he said: "The great war is between death and life, ice and fire. If we loose, the night will never end"<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;It has been six months since the white walkers' army marched into the north, led by the night king himself on a dead dragon. It has been a battle like never before: never before have men faced such an enemy in battle, never before have men fought so bravely against a united threat, and never before have they been so gravely defeated.<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; While Cersei is in King's landing, brave men have died fighting the great war. Among others, Tyrion is dead, Arya is dead and Jon Snow is dead, again. In a desperate battle, Daenerys leads all her forces in a final stand-off with the dead just south of Winterfell. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Her army defeated, she is now on the run on her dragon in an air battle, being chased by two of her own dragons, the Night king and a dead Jon Snow. Suddenly, the Night king's spear hits Danny's dragon, who, raining blood and fire, falls into ice, taking the lost queen, with him. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Daenerys opens her eyes in a strange place, a place which does not follow the rules of space and time, where the dead souls killed by the dead men are trapped, forever. But who woke her up? There stands near her, Tyrion, with Jorah, Davos, Jon Snow, and everybody else. They all indulge in a heartfelt reunion when someone yells- "But how do we get out?<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Varys sees a talking crystal close by, who asks them of completing a task, which on completion would allow them to go back to the land of the living, with the ultimate tool to defeat the white-walkers and kills the night king, the Dragon-axe. They have summoned you for help, as the task is out of their expertise, to apply a modified CNN to solve the object detection problem on the PASCAL VOC dataset. Varys, the master of whisperers, has used his talents to import the following for you:

In [37]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()
# You can ask Varys to get you more if you desire
import xml.etree.ElementTree as ET
import glob
import PIL.Image
import PIL.ImageChops
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
import torchvision
import skimage.measure
import skimage.morphology
from random import randint

resnet_input = 224#size of resnet18 input images

In [38]:
# Cersei chose violence, you choose your hyper-parameters wisely using validation data!
batch_size = 2
num_epochs = 5
learning_rate =  0.001
hyp_momentum = 0.9

## Build the data
The hound who was in charge for getting the data, brought you the following links:
<br/>Training and validation:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
<br/>Testing data:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
<br/>He also told you that the dataset(datascrolls :P) consists of images from of 20 classes, with detection annotations included. The JPEGImages folder houses the images, and the Annotations folder has the object-wise labels for the objects in one xml file per image. You have to extract the object information, ie. the [xmin, ymin] (the top left x,y co-ordinates) and the [xmax, ymax] (the bottom right x,y co-ordinates) of only the objects belonging to the given 20 classes(aeroplane, bicycle, boat, bottle, bus, car, cat, chair, cow, dining table, dog, horse, motorbike, person, potted plant, sheep, train, TV). For parsing the xml file, you can ask Varys to import xml.etree.ElementTree for you. <br/>
<br/> You can then ask Bronn and Jamie to organize the data as follows:
<br/> For every image in the dataset, extract/crop the object patch from the image one by one using their respective co-ordinates:[xmin, ymin, xmax, ymax], resize the image to resnet_input, and store it with its class label information. Do the same for training/validation and test datasets. <br/>
##### Important
You also have to collect data for an extra background class which stands for the class of an object which is not a part of any of the 20 classes. For this, you can crop and resize any random patches from an image. A good idea is to extract patches that have low "intersection over union" with any object present in the image frame from the 20 Pascal VOC classes. The number of background images should be roughly around those of other class objects' images. Hence the total classes turn out to be 21. This is important for applying the sliding window method later.

In [39]:
classes = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')


In [40]:
# Taken form pyimagesearch for calculating intersection over union
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # compute the area of intersection rectangle
    interArea = (xB - xA + 1) * (yB - yA + 1)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea + np.finfo(float).eps)
 
    # return the intersection over union value
    return iou

In [41]:
# Taken from stackoverflow for drawing random bounding boxes
def random_bbox(bbox):
    v = [randint(0, v) for v in bbox]
    left = min(v[0], v[2])
    upper = min(v[1], v[3])
    right = max(v[0], v[2])
    lower = max(v[1], v[3])
    return [left, upper, right, lower]

In [42]:
def jamie_bronn_build_dataset(dir,img_path):            
    image = PIL.Image.open(img_path)
    img_name = img_path.split("/")[-1].split(".")[-2]
    print(img_name)
    xml_path = dir+'/Annotations/'+img_name+'.xml'
    xml_tree = ET.parse(xml_path)
    xml_root = xml_tree.getroot()

    temp_img = PIL.Image.new('RGB',image.size,0)
#     image.show()
    
    location = []
    labels = []
    object_img = []
    for object in xml_root.findall('object'):
        name = object.find('name').text
        position = [int(object.find('bndbox').find('xmin').text), int(object.find('bndbox').find('ymin').text),
                    int(object.find('bndbox').find('xmax').text), int(object.find('bndbox').find('ymax').text)]
        location.append(position)
        crop_img = image.crop(position).convert('RGB')
        object_img.append(crop_img)
        labels.append(classes.index(name))
        
#         temp_img.paste(crop_img,position)

#     temp_img = PIL.ImageChops.subtract(image,temp_img)
#     l = skimage.morphology.label(np.array(image.convert('L')))
#     regions = skimage.measure.regionprops(l)
#     max_area = 0
#     for region in regions:
#         if region.area >= max_area:
#             position = region.bbox
#             max_area = region.area
            
#     location.append(position)        
#     crop_img = image.crop(position).convert('RGB')
# #     crop_img.show()
#     object_img.append(crop_img)
#     labels.append(classes.index('__background__'))

    iou_threshold = 0.3
    num = 0
    while(num < 1):
        bbox = image.getbbox()
        boxA = random_bbox(bbox)
        mscore = 0
        for boxB in location:
              score = (bb_intersection_over_union(boxA, boxB))
              if (score > mscore):
                  mscore = score
        if (mscore < iou_threshold):
            object_img.append(image.crop(boxA).convert('RGB'))
            labels.append(classes.index('__background__'))
            num = num + 1
        
    return object_img, labels

In [43]:
class hound_dataset(torch.utils.data.Dataset): # Extend PyTorch's Dataset class
    def __init__(self, root_dir, train, transform=None):
        # Begin
        print(train)
        if(train):
            dir = root_dir + '/train/VOCdevkit/VOC2007'
        else :
            dir = root_dir + '/test/VOCdevkit/VOC2007'
        self.transform = transform
        self.img = [];
        self.label = [];
        i = 0
        for img_path in glob.glob(dir+'/JPEGImages/*.jpg'):
            object_img, name = jamie_bronn_build_dataset(dir,img_path)
            self.img.extend(object_img)
            self.label.extend(name)
#             i = i+1
#             if i == 7:
#                 break
                       
                
    def __len__(self):
        return len(self.img)
        
    def __getitem__(self, idx):
        if self.transform is None:
            return (self.img[idx],self.label[idx])
        else:
            img_transformed = self.transform(self.img[idx])
            return (img_transformed,self.label[idx])

In [44]:
# train_dataset = hound_dataset(root_dir='.', train=False, transform=None) # Supply proper root_dir

## Train the netwok
<br/>You can ask Arya to train the network on the created dataset. This will yield a classification network on the 21 classes of the VOC dataset. 

In [45]:
composed_transform = transforms.Compose([transforms.Scale((resnet_input,resnet_input)),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor()])
train_dataset = hound_dataset(root_dir='.', train=True, transform=composed_transform) # Supply proper root_dir
# test_dataset = hound_dataset(root_dir='.', train=False, transform=composed_transform) # Supply proper root_dir

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

True
008310
001492
004514
004611
003239
004685
000806
008390
002933
003699
001137
002520
007803
007330
002056
005173
004512
006346
008100
007311
002036
003382
007997
005058
009764
006209
003790
003280
001486
003618
003646
002253
008213
006357
007141
008771
000977
007970
008076
009745
009905
004111
002272
004186
000524
006398
000352
006424
002180
004228
003783
005217
000164
006468
008360
004019
003337
008982
008529
007897
005845
009532
003299
007292
001755
000270
002899
005851
004140
004607
004926
003135
009299
004423
006725
006852
007544
001442
001504
000477
006687
005186
003039
003824
009834
007208
007132
001522
002345
005487
000435
005086
006355
006179
002140
002858
005930
008426
007138
004033
003821
008318
005358
000867
008069
000215
000608
003654
005094
001677
002969
007745
007370
001523
004649
006035
003094
006133
006304
005517
002405
000308
007474
006275
007398
000726
000851
001068
003017
001012
006919
009285
009945
008253
003260
008268
005897
006314
005169
001239
004999
001290
0

001441
006208
000892
003767
004145
006128
001014
001259
006802
004195
006949
002145
000464
002218
008093
009836
007052
003857
004307
000275
002437
003159
007097
009108
001555
009209
007890
000516
004644
009131
000794
004396
000748
009039
003466
002477
001877
004169
004576
000592
000081
009271
007621
007446
005352
005856
005144
007470
002954
008843
005278
005796
002944
001121
005404
002324
006417
001024
000946
004121
005542
006098
001205
001978
008542
000480
008425
007080
007077
007727
007350
003425
000934
005652
000782
002967
003462
006610
000047
008346
000973
007753
009189
009463
004911
006874
004468
007667
007831
006706
008653
004674
001409
003021
006538
008190
003218
000964
004077
004253
006335
003023
005786
004552
004687
006840
003895
002912
008106
006011
009586
003997
003117
003567
009908
008663
008801
008115
001084
009807
008068
004708
007876
004524
005388
007475
006965
000259
008125
005700
000965
009585
000899
006681
007125
006562
002891
009278
000276
006055
000514
007558
003828

009290
009476
007678
002481
003338
006682
009868
003335
001544
005911
006519
001593
006849
002126
004452
003721
008988
004579
002632
001176
005825
002311
004441
001045
008096
006523
000060
004627
001127
002067
003038
002932
004474
002228
004110
001684
004281
007672
000407
002139
007040
006933
003849
001603
001827
008098
006214
004025
008250
000321
008596
004998
000417
001948
007521
004544
000625
008044
007211
009613
004439
003949
000367
003435
009832
007611
002611
006046
005408
006696
008576
008942
004239
004170
000997
002571
004013
004584
001136
009792
005655
003057
001982
002682
006570
009823
007411
006512
007498
002836
006171
006436
007230
003603
003202
008699
006103
000882
005910
008755
007493
004595
004588
007007
002212
008509
009810
009163
005398
006369
005519
003548
005918
004384
003412
006074
000233
003045
006548
009940
002884
003108
007058
004912
009221
008258
009091
002637
007197
004872
007712
009016
002810
001406
008433
005536
001537
009406
008040
003491
006134
000800
005784

005108
005917
000672
005268
000035
000102
001057
000827
006206
001882
004148
001772
003477
004948
000688
006734
003996
006605
006483
006158
006425
009551
007385
005336
004783
001577
004836
004095
004655
004273
000713
003740
007916
009580
007626
004898
000786
005773
009707
000430
007841
003377
002334
003499
006241
000214
005033
004793
009755
002461
002605
003138
007486
001862
000537
007374
003605
006330
006170
007250
006071
000042
006488
003886
009568
002102
003339
003555
003034
003205
000460
004785
001288
004075
000189
009761
002221
001263
007422
001330
008624
007210
002315
003784
007182
006004
002494
005136
006626
009448
009351
005239
006833
004164
008815
009412
008790
002004
007819
008862
002827
004303
007458
001999
005461
003609
007723
006462
004812
003009
006740
007184
001780
005037
002201
001027
002364
006842
009781
003455
001451
007489
001972
001557
009800
001699
004574
007868
002646
004275
007477
002375
005579
009687
008477
007513
006033
008930
008387
008753
005588
008140
006971

001766
009382
008919
007525
003253
002023
005593
008716
003759
000764
001479
004012
000710
001432
009584
003401
007108
006755
006140
005387
000406
005826
001304
004732
002458
000424
006805
000544
007021
002590
008939
002376
006990
002328
001746
006994
001184
003183
009195
000208
006156
000597
006730
009179
004020
001093
003924
004519
004372
005760
003593
008971
001954
004333
008062
005609
003126
002866
001148
003380
006222
004693
008438
005630
006524
009087
006709
003957
001761
003594
006058
005738
006448
006230
009259
008322
009073
003988
008009
006107
002448
001821
009588
003596
003149
006088
009519
005349
005326
008568
004782
005209
000077
005003
004660
008517
007065
004823
008150
007336
009700
007933
000796
008413
001388
007296
002392
007572
007799
002989
002146
006569
006738
002491
008606
007185
009470
006847
003303
005304
001756
001536
006163
009671
002330
005470
007720
005496
005303
008053
005824
000416
007662
000850
003325
004939
005740
009245
003397
009334
004837
001395
009218

### Fine-tuning
Litlefinger has brought you a pre-trained network. Fine-tune the network in the following section:

In [46]:
resnet18 = models.resnet18(pretrained=True)

resnet18.fc = nn.Linear(resnet18.fc.in_features, 21)

# Add code for using CUDA here
use_gpu = False
if(torch.cuda.is_available()):
    use_gpu = True
    resnet18.cuda()

In [47]:
criterion = nn.CrossEntropyLoss()
# Update if any errors occur
optimizer = torch.optim.SGD(resnet18.parameters(), learning_rate, hyp_momentum)

In [48]:
def arya_train():
    # Begin
    loss_arr = []
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):  
            # Convert torch tensor to Variable
            images = Variable(images)
            labels = Variable(labels)
            if(use_gpu):
                images=images.cuda()
                labels=labels.cuda()
            # Forward + Backward + Optimize
            optimizer.zero_grad()  # zero the gradient buffer
            outputs = resnet18(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss_arr.append(loss.data[0])
            if (i+1) % batch_size == 0:
                print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                       %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
    plt.plot( np.array(range(1,len(loss_arr)+1)), np.array(loss_arr))
    plt.show()

In [ ]:
%time arya_train()
torch.save(resnet18.state_dict(), 'resnet18.pkl')

Epoch [1/5], Step [2/10336], Loss: 3.1157
Epoch [1/5], Step [4/10336], Loss: 3.1148
Epoch [1/5], Step [6/10336], Loss: 1.2156
Epoch [1/5], Step [8/10336], Loss: 2.6169
Epoch [1/5], Step [10/10336], Loss: 2.1820
Epoch [1/5], Step [12/10336], Loss: 5.2420
Epoch [1/5], Step [14/10336], Loss: 1.4026
Epoch [1/5], Step [16/10336], Loss: 3.1209
Epoch [1/5], Step [18/10336], Loss: 2.3037
Epoch [1/5], Step [20/10336], Loss: 3.1656
Epoch [1/5], Step [22/10336], Loss: 5.3041
Epoch [1/5], Step [24/10336], Loss: 3.9647
Epoch [1/5], Step [26/10336], Loss: 2.3611
Epoch [1/5], Step [28/10336], Loss: 3.7858
Epoch [1/5], Step [30/10336], Loss: 2.4816
Epoch [1/5], Step [32/10336], Loss: 4.4216
Epoch [1/5], Step [34/10336], Loss: 2.8029
Epoch [1/5], Step [36/10336], Loss: 1.7905
Epoch [1/5], Step [38/10336], Loss: 3.5114
Epoch [1/5], Step [40/10336], Loss: 4.8595
Epoch [1/5], Step [42/10336], Loss: 1.9893
Epoch [1/5], Step [44/10336], Loss: 2.3079
Epoch [1/5], Step [46/10336], Loss: 4.2023
Epoch [1/5], St

Epoch [1/5], Step [378/10336], Loss: 0.4406
Epoch [1/5], Step [380/10336], Loss: 4.9114
Epoch [1/5], Step [382/10336], Loss: 0.7768
Epoch [1/5], Step [384/10336], Loss: 3.1874
Epoch [1/5], Step [386/10336], Loss: 1.7735
Epoch [1/5], Step [388/10336], Loss: 2.0891
Epoch [1/5], Step [390/10336], Loss: 2.3576
Epoch [1/5], Step [392/10336], Loss: 3.7717
Epoch [1/5], Step [394/10336], Loss: 0.2124
Epoch [1/5], Step [396/10336], Loss: 4.4715
Epoch [1/5], Step [398/10336], Loss: 0.3438
Epoch [1/5], Step [400/10336], Loss: 1.0244
Epoch [1/5], Step [402/10336], Loss: 1.9453
Epoch [1/5], Step [404/10336], Loss: 0.2580
Epoch [1/5], Step [406/10336], Loss: 2.3346
Epoch [1/5], Step [408/10336], Loss: 2.4306
Epoch [1/5], Step [410/10336], Loss: 1.3865
Epoch [1/5], Step [412/10336], Loss: 0.9417
Epoch [1/5], Step [414/10336], Loss: 1.2873
Epoch [1/5], Step [416/10336], Loss: 1.0630
Epoch [1/5], Step [418/10336], Loss: 0.8240
Epoch [1/5], Step [420/10336], Loss: 0.8855
Epoch [1/5], Step [422/10336], L

Epoch [1/5], Step [752/10336], Loss: 3.5866
Epoch [1/5], Step [754/10336], Loss: 0.9890
Epoch [1/5], Step [756/10336], Loss: 2.4233
Epoch [1/5], Step [758/10336], Loss: 1.8693
Epoch [1/5], Step [760/10336], Loss: 3.1513
Epoch [1/5], Step [762/10336], Loss: 1.4006
Epoch [1/5], Step [764/10336], Loss: 2.7045
Epoch [1/5], Step [766/10336], Loss: 2.0460
Epoch [1/5], Step [768/10336], Loss: 3.7936
Epoch [1/5], Step [770/10336], Loss: 3.7918
Epoch [1/5], Step [772/10336], Loss: 5.0932
Epoch [1/5], Step [774/10336], Loss: 1.7390
Epoch [1/5], Step [776/10336], Loss: 2.8166
Epoch [1/5], Step [778/10336], Loss: 2.4839
Epoch [1/5], Step [780/10336], Loss: 2.9437
Epoch [1/5], Step [782/10336], Loss: 1.8907
Epoch [1/5], Step [784/10336], Loss: 3.1223
Epoch [1/5], Step [786/10336], Loss: 1.2076
Epoch [1/5], Step [788/10336], Loss: 5.7973
Epoch [1/5], Step [790/10336], Loss: 2.8630
Epoch [1/5], Step [792/10336], Loss: 3.4587
Epoch [1/5], Step [794/10336], Loss: 2.5158
Epoch [1/5], Step [796/10336], L

Epoch [1/5], Step [1122/10336], Loss: 2.0063
Epoch [1/5], Step [1124/10336], Loss: 2.2194
Epoch [1/5], Step [1126/10336], Loss: 6.9224
Epoch [1/5], Step [1128/10336], Loss: 3.7532
Epoch [1/5], Step [1130/10336], Loss: 3.0403
Epoch [1/5], Step [1132/10336], Loss: 0.6285
Epoch [1/5], Step [1134/10336], Loss: 2.6102
Epoch [1/5], Step [1136/10336], Loss: 6.3338
Epoch [1/5], Step [1138/10336], Loss: 2.3787
Epoch [1/5], Step [1140/10336], Loss: 2.2145
Epoch [1/5], Step [1142/10336], Loss: 1.4516
Epoch [1/5], Step [1144/10336], Loss: 3.7452
Epoch [1/5], Step [1146/10336], Loss: 0.3589
Epoch [1/5], Step [1148/10336], Loss: 1.1305
Epoch [1/5], Step [1150/10336], Loss: 1.4301
Epoch [1/5], Step [1152/10336], Loss: 1.3189
Epoch [1/5], Step [1154/10336], Loss: 0.9081
Epoch [1/5], Step [1156/10336], Loss: 4.7610
Epoch [1/5], Step [1158/10336], Loss: 1.3323
Epoch [1/5], Step [1160/10336], Loss: 5.3369
Epoch [1/5], Step [1162/10336], Loss: 0.1775
Epoch [1/5], Step [1164/10336], Loss: 2.4945
Epoch [1/5

Epoch [1/5], Step [1488/10336], Loss: 3.9884
Epoch [1/5], Step [1490/10336], Loss: 3.6718
Epoch [1/5], Step [1492/10336], Loss: 2.1198
Epoch [1/5], Step [1494/10336], Loss: 1.5712
Epoch [1/5], Step [1496/10336], Loss: 0.7720
Epoch [1/5], Step [1498/10336], Loss: 2.1193
Epoch [1/5], Step [1500/10336], Loss: 0.0851
Epoch [1/5], Step [1502/10336], Loss: 1.1124
Epoch [1/5], Step [1504/10336], Loss: 2.9827
Epoch [1/5], Step [1506/10336], Loss: 5.9569
Epoch [1/5], Step [1508/10336], Loss: 0.7377
Epoch [1/5], Step [1510/10336], Loss: 0.7324
Epoch [1/5], Step [1512/10336], Loss: 0.4417
Epoch [1/5], Step [1514/10336], Loss: 2.2304
Epoch [1/5], Step [1516/10336], Loss: 1.1195
Epoch [1/5], Step [1518/10336], Loss: 5.9763
Epoch [1/5], Step [1520/10336], Loss: 0.1263
Epoch [1/5], Step [1522/10336], Loss: 1.7335
Epoch [1/5], Step [1524/10336], Loss: 2.7407
Epoch [1/5], Step [1526/10336], Loss: 3.7678
Epoch [1/5], Step [1528/10336], Loss: 0.2008
Epoch [1/5], Step [1530/10336], Loss: 2.4045
Epoch [1/5

...

Epoch [1/5], Step [2586/10336], Loss: 2.5525
Epoch [1/5], Step [2588/10336], Loss: 2.2055
Epoch [1/5], Step [2590/10336], Loss: 1.3160
Epoch [1/5], Step [2592/10336], Loss: 0.1987
Epoch [1/5], Step [2594/10336], Loss: 0.9521
Epoch [1/5], Step [2596/10336], Loss: 4.3784
Epoch [1/5], Step [2598/10336], Loss: 1.6013
Epoch [1/5], Step [2600/10336], Loss: 0.3167
Epoch [1/5], Step [2602/10336], Loss: 2.4959
Epoch [1/5], Step [2604/10336], Loss: 1.7241
Epoch [1/5], Step [2606/10336], Loss: 2.1510
Epoch [1/5], Step [2608/10336], Loss: 0.3092
Epoch [1/5], Step [2610/10336], Loss: 0.8564
Epoch [1/5], Step [2612/10336], Loss: 3.4814
Epoch [1/5], Step [2614/10336], Loss: 0.2292
Epoch [1/5], Step [2616/10336], Loss: 2.7915
Epoch [1/5], Step [2618/10336], Loss: 1.5133
Epoch [1/5], Step [2620/10336], Loss: 2.3190
Epoch [1/5], Step [2622/10336], Loss: 4.9105
Epoch [1/5], Step [2624/10336], Loss: 1.9943
Epoch [1/5], Step [2626/10336], Loss: 0.9368
Epoch [1/5], Step [2628/10336], Loss: 2.4417
Epoch [1/5

Epoch [1/5], Step [2952/10336], Loss: 1.4867
Epoch [1/5], Step [2954/10336], Loss: 4.7482
Epoch [1/5], Step [2956/10336], Loss: 1.2304
Epoch [1/5], Step [2958/10336], Loss: 0.8887
Epoch [1/5], Step [2960/10336], Loss: 2.7847
Epoch [1/5], Step [2962/10336], Loss: 0.6927
Epoch [1/5], Step [2964/10336], Loss: 2.5848
Epoch [1/5], Step [2966/10336], Loss: 1.0322
Epoch [1/5], Step [2968/10336], Loss: 3.7295
Epoch [1/5], Step [2970/10336], Loss: 0.2351
Epoch [1/5], Step [2972/10336], Loss: 1.1534
Epoch [1/5], Step [2974/10336], Loss: 3.5203
Epoch [1/5], Step [2976/10336], Loss: 0.5021
Epoch [1/5], Step [2978/10336], Loss: 0.3655
Epoch [1/5], Step [2980/10336], Loss: 3.7944
Epoch [1/5], Step [2982/10336], Loss: 1.8792
Epoch [1/5], Step [2984/10336], Loss: 2.3440
Epoch [1/5], Step [2986/10336], Loss: 1.4932
Epoch [1/5], Step [2988/10336], Loss: 1.3569
Epoch [1/5], Step [2990/10336], Loss: 2.4441
Epoch [1/5], Step [2992/10336], Loss: 3.4563
Epoch [1/5], Step [2994/10336], Loss: 1.2959
Epoch [1/5

Epoch [1/5], Step [3318/10336], Loss: 0.8535
Epoch [1/5], Step [3320/10336], Loss: 3.8826
Epoch [1/5], Step [3322/10336], Loss: 0.6437
Epoch [1/5], Step [3324/10336], Loss: 1.4520
Epoch [1/5], Step [3326/10336], Loss: 3.3939
Epoch [1/5], Step [3328/10336], Loss: 0.4063
Epoch [1/5], Step [3330/10336], Loss: 0.4223
Epoch [1/5], Step [3332/10336], Loss: 2.4365
Epoch [1/5], Step [3334/10336], Loss: 6.4639
Epoch [1/5], Step [3336/10336], Loss: 1.6450
Epoch [1/5], Step [3338/10336], Loss: 0.4076
Epoch [1/5], Step [3340/10336], Loss: 2.1326
Epoch [1/5], Step [3342/10336], Loss: 0.6504
Epoch [1/5], Step [3344/10336], Loss: 1.4700
Epoch [1/5], Step [3346/10336], Loss: 1.8374
Epoch [1/5], Step [3348/10336], Loss: 0.7315
Epoch [1/5], Step [3350/10336], Loss: 0.5170
Epoch [1/5], Step [3352/10336], Loss: 1.6257
Epoch [1/5], Step [3354/10336], Loss: 2.1153
Epoch [1/5], Step [3356/10336], Loss: 2.3656
Epoch [1/5], Step [3358/10336], Loss: 1.8682
Epoch [1/5], Step [3360/10336], Loss: 0.9103
Epoch [1/5

Epoch [1/5], Step [3684/10336], Loss: 3.1734
Epoch [1/5], Step [3686/10336], Loss: 1.5576
Epoch [1/5], Step [3688/10336], Loss: 1.7151
Epoch [1/5], Step [3690/10336], Loss: 2.2463
Epoch [1/5], Step [3692/10336], Loss: 3.7830
Epoch [1/5], Step [3694/10336], Loss: 2.1940
Epoch [1/5], Step [3696/10336], Loss: 1.8296
Epoch [1/5], Step [3698/10336], Loss: 2.7311
Epoch [1/5], Step [3700/10336], Loss: 1.3752
Epoch [1/5], Step [3702/10336], Loss: 3.0272
Epoch [1/5], Step [3704/10336], Loss: 1.7613
Epoch [1/5], Step [3706/10336], Loss: 0.5125
Epoch [1/5], Step [3708/10336], Loss: 0.9084
Epoch [1/5], Step [3710/10336], Loss: 1.9691
Epoch [1/5], Step [3712/10336], Loss: 1.1990
Epoch [1/5], Step [3714/10336], Loss: 0.3424
Epoch [1/5], Step [3716/10336], Loss: 1.1254
Epoch [1/5], Step [3718/10336], Loss: 0.2961
Epoch [1/5], Step [3720/10336], Loss: 4.9947
Epoch [1/5], Step [3722/10336], Loss: 3.6740
Epoch [1/5], Step [3724/10336], Loss: 0.9192
Epoch [1/5], Step [3726/10336], Loss: 1.6490
Epoch [1/5

Epoch [1/5], Step [4050/10336], Loss: 1.3782
Epoch [1/5], Step [4052/10336], Loss: 0.9502
Epoch [1/5], Step [4054/10336], Loss: 1.3135
Epoch [1/5], Step [4056/10336], Loss: 1.1169
Epoch [1/5], Step [4058/10336], Loss: 0.8978
Epoch [1/5], Step [4060/10336], Loss: 1.9317
Epoch [1/5], Step [4062/10336], Loss: 0.0682
Epoch [1/5], Step [4064/10336], Loss: 0.6641
Epoch [1/5], Step [4066/10336], Loss: 0.2973
Epoch [1/5], Step [4068/10336], Loss: 0.5948
Epoch [1/5], Step [4070/10336], Loss: 0.2732
Epoch [1/5], Step [4072/10336], Loss: 1.9982
Epoch [1/5], Step [4074/10336], Loss: 2.4418
Epoch [1/5], Step [4076/10336], Loss: 3.1141
Epoch [1/5], Step [4078/10336], Loss: 1.2858
Epoch [1/5], Step [4080/10336], Loss: 0.3725
Epoch [1/5], Step [4082/10336], Loss: 0.6011
Epoch [1/5], Step [4084/10336], Loss: 4.2858
Epoch [1/5], Step [4086/10336], Loss: 4.7459
Epoch [1/5], Step [4088/10336], Loss: 0.0750
Epoch [1/5], Step [4090/10336], Loss: 2.1931
Epoch [1/5], Step [4092/10336], Loss: 1.0740
Epoch [1/5

# Testing and Accuracy Calculation
Jorah then asks a question, how is this a detection task?<br/>
As everybody wonders, Theon Greyjoy suggests a slding window method to test the above trained trained network on the detection task:<br/>
"We take some windows of varying size and aspect ratios", he mumbled, "and slide it through the test image (considering some stride of pixels) from left to right, and top to bottom, detect the class scores for each of the window, and keep only those which are above a certain threshold value!". "He is right", says Samwell, "I read a similar approach in the paper -Faster RCNN by Ross Girshick in the library, where he uses three diferent scales/sizes and three different aspect ratios, making a total of nine windows per pixel to slide". You need to write the code and use it in testing code to find the predicted boxes and their classes.

In [ ]:
def theon_sliding_window(image,aspect_ratio,size,slide_amount):
    # Begin
    windows = []
    for x in xrange(0,image.size(0),slide_amount):
        for y in xrange(0,image.size(1),slide_amount):
            box = [x,y,x+size*aspect_ratio,y+size]
            crop_img = image.crop(box).convert('RGB')
            output = resnet18(composed_transform(crop_img))
#             if(output.data)
    return output

"Wait", says <b>Jon Snow</b>, "The predicted boxes may be too many and we can't deal with all of them. So, I myself will go and apply non_maximum_supression to reduce the number of boxes". You are free to choose the threshold value for non maximum supression, but choose wisely [0,1].

In [ ]:
def aegon_targaryen_non_maximum_supression(boxes,threshold = 0.3):
    # 

Daenerys, the queen, then orders her army to test out the trained model on the test dataset.

In [ ]:
def daenerys_test(resnet18):
    # Write loops for testing the model on the test set
    # Also print out the accuracy of the model

In [ ]:
%time daenerys_test(resnet18)

# Final Showdown
After covering all the steps and passing the accuracy value to the talking crystal, they all pass through to the land of the living, with a wounded Jon Snow armed with the Dragon-axe. After a fierce battle, Jon Snow manages to go face to face with the Night king. Surrounded by battling men and falling bodies, they engage in a ferocious battle, a battle of spear and axe. After a raging fight, Jon manages to sink the axe into the Night king's heart, but not before he gets wounded by the spear. As dead men fall to bones, Daenerys and others rush to his aid, but it is too late. Everyone is in tears as they look towards the man of honour, Jon Snow, lying in Daenerys's arms when he says his last words: "The night has ended. Winter is finally over!"

In [ ]:
print("The night has ended. Winter is finally over!")